# Neural networks

In [1]:
# Imports and setup
import gensim
import multiprocessing as mp
import nltk
import numpy as np
import os
import pickle
import sqlite3
import unicodedata

from gensim.matutils import sparse2full, full2sparse, full2sparse_clipped, scipy2scipy_clipped
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from itertools import groupby
from keras.layers import Dense, Dropout, Activation, LSTM
from keras.layers.embeddings import Embedding
from keras.models import load_model, Sequential
from keras.preprocessing import sequence
from keras.wrappers.scikit_learn import KerasClassifier
from nltk.corpus import wordnet as wn
from nltk.corpus.reader.api import CorpusReader
from nltk.chunk import tree2conlltags
from nltk.probability import FreqDist
from nltk.chunk.regexp import RegexpParser
from nltk.stem.wordnet import WordNetLemmatizer
from random import choice
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.externals import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelBinarizer
from slugify import slugify
from unicodedata import category as unicat

# Local paths
pitchfork_dir = os.path.join('..', 'data', 'pitchfork')
sql_file = os.path.join(pitchfork_dir, 'database.sqlite')
pickle_dir = os.path.join(pitchfork_dir, 'pickled')
pitchfork_model = os.path.join(pitchfork_dir, 'pitchfork_clf.pkl')

# Settings
preproc = False # Perform preprocessing? (takes c. 1.5 hours)

Using TensorFlow backend.


## Basics: Perceptron

In [2]:
np.random.seed(1)

# Neuron activates if x is positive
activation = lambda x: 0 if x < 0 else 1

# 3D input data with labels
training_data = [(np.array([0,0,1]), 0),
                 (np.array([0,1,1]), 0),
                 (np.array([1,0,1]), 0),
                 (np.array([1,1,1]), 1),
                ]

# Model parameters
learning_rate = 0.2 
training_steps = 1000

# Initialize weights to random 3D arrays
# Values range from zero to one
W = np.random.rand(3) 

# Perform the indicated number of training steps
for i in range(training_steps):
    
    x, y = choice(training_data) # Pick one data element on which to train
    
    layer1 = np.dot(W, x)        # Compute the output weight
    y_pred = activation(layer1)  # Pass output weight to activation, i.e., prediction
    
    error = y - y_pred           # Calculate amount of error (here, 0 or 1)
    update = learning_rate * error * x # Calculate update weight
    W += update                  # Update array of weights

# Output after training
print("Predictions after training")
for x, _ in training_data:
    y_pred = np.dot(x, W)
    print("{}: {}".format(x[:2], activation(y_pred)))

Predictions after training
[0 0]: 0
[0 1]: 0
[1 0]: 0
[1 1]: 1


## Sentiment analysis with Scikit-learn

Predict album review sentiment from the [Kaggle Pitchfork dataset](https://www.kaggle.com/nolanbconaway/pitchfork-data/data).

Need to install three new libraries for this section and the following ones:

```
conda install keras tensorflow python-slugify
```

### Readers and utilities

In [3]:
PKL_PATTERN = r'(?!\.)[\w\s\d\-]+\.pickle'

class SqliteCorpusReader(object):

    def __init__(self, path):
        self._cur = sqlite3.connect(path).cursor()

    def scores(self):
        """
        Returns the review score
        """
        self._cur.execute("SELECT score FROM reviews")
        scores = self._cur.fetchall()
        for score in scores:
            yield score

    def texts(self):
        """
        Returns the full review texts
        """
        self._cur.execute("SELECT content FROM content")
        texts = self._cur.fetchall()
        for text in texts:
            yield text

    def ids(self):
        """
        Returns the review ids
        """
        self._cur.execute("SELECT reviewid FROM content")
        ids = self._cur.fetchall()
        for idx in ids:
            yield idx

    def ids_and_texts(self):
        """
        Returns the review ids
        """
        self._cur.execute("SELECT * FROM content")
        results = self._cur.fetchall()
        for idx,text in results:
            yield idx,text

    def scores_albums_artists_texts(self):
        """
        Returns a generator with each review represented as a
        (score, album name, artist name, review text) tuple
        """
        sql = """
              SELECT S.score, L.label, A.artist, R.content
              FROM [reviews] S
              JOIN labels L ON S.reviewid=L.reviewid
              JOIN artists A on L.reviewid=A.reviewid
              JOIN content R ON A.reviewid=R.reviewid
              """
        self._cur.execute(sql)
        results = self._cur.fetchall()
        for score,album,band,text in results:
            yield (score,album,band,text)

    def albums(self):
        """
        Returns the names of albums being reviewed
        """
        self._cur.execute("SELECT * FROM labels")
        albums = self._cur.fetchall()
        for idx,album in albums:
            yield idx,album

    def artists(self):
        """
        Returns the name of the artist being reviewed
        """
        self._cur.execute("SELECT * FROM artists")
        artists = self._cur.fetchall()
        for idx,artist in artists:
            yield idx,artist

    def genres(self):
        """
        Returns the music genre of each review
        """
        self._cur.execute("SELECT * FROM genres")
        genres = self._cur.fetchall()
        for idx,genre in genres:
            yield idx,genre

    def years(self):
        """
        Returns the publication year of each review

        Note: There are many missing values
        """
        self._cur.execute("SELECT * FROM years")
        years = self._cur.fetchall()
        for idx,year in years:
            yield idx,year

    def paras(self):
        """
        Returns a generator of paragraphs.
        """
        for text in self.texts():
            for paragraph in text:
                yield paragraph

    def sents(self):
        """
        Returns a generator of sentences.
        """
        for para in self.paras():
            for sentence in nltk.sent_tokenize(para):
                yield sentence

    def words(self):
        """
        Returns a generator of words.
        """
        for sent in self.sents():
            for word in nltk.wordpunct_tokenize(sent):
                yield word

    def tagged_tokens(self):
        for sent in self.sents():
            for word in nltk.wordpunct_tokenize(sent):
                yield nltk.pos_tag(word)



class PickledReviewsReader(CorpusReader):
    def __init__(self, root, fileids=PKL_PATTERN, **kwargs):
        """
        Initialize the corpus reader
        """
        CorpusReader.__init__(self, root, fileids, **kwargs)

    def texts_scores(self, fileids=None):
        """
        Returns the document loaded from a pickled object for every file in
        the corpus. Similar to the SqliteCorpusReader, this uses a generator
        to achieve memory safe iteration.
        """
        # Create a generator, loading one document into memory at a time.
        for path, enc, fileid in self.abspaths(fileids, True, True):
            with open(path, 'rb') as f:
                yield pickle.load(f)

    def reviews(self, fileids=None):
        """
        Returns a generator of paragraphs where each paragraph is a list of
        sentences, which is in turn a list of (token, tag) tuples.
        """
        for text,score in self.texts_scores(fileids):
            yield text

    def scores(self, fileids=None):
        """
        Return the scores
        """
        for text,score in self.texts_scores(fileids):
            yield score

    def paras(self, fileids=None):
        """
        Returns a generator of paragraphs where each paragraph is a list of
        sentences, which is in turn a list of (token, tag) tuples.
        """
        for review in self.reviews(fileids):
            for paragraph in review:
                yield paragraph

    def sents(self, fileids=None):
        """
        Returns a generator of sentences where each sentence is a list of
        (token, tag) tuples.
        """
        for paragraph in self.paras(fileids):
            for sentence in paragraph:
                yield sentence

    def tagged(self, fileids=None):
        for sent in self.sents(fileids):
            for token in sent:
                yield token

    def words(self, fileids=None):
        """
        Returns a generator of (token, tag) tuples.
        """
        for token in self.tagged(fileids):
            yield token[0]

### Transformers

In [4]:
class TextNormalizer(BaseEstimator, TransformerMixin):

    def __init__(self, language='english'):
        self.stopwords  = set(nltk.corpus.stopwords.words(language))
        self.lemmatizer = WordNetLemmatizer()

    def is_punct(self, token):
        return all(
            unicodedata.category(char).startswith('P') for char in token
        )

    def is_stopword(self, token):
        return token.lower() in self.stopwords

    def normalize(self, document):
        return [
            self.lemmatize(token, tag).lower()
            for sentence in document
            for (token, tag) in sentence
            if not self.is_punct(token)
               and not self.is_stopword(token)
        ]

    def lemmatize(self, token, pos_tag):
        tag = {
            'N': wn.NOUN,
            'V': wn.VERB,
            'R': wn.ADV,
            'J': wn.ADJ
        }.get(pos_tag[0], wn.NOUN)

        return self.lemmatizer.lemmatize(token, tag)

    def fit(self, documents, y=None):
        return self

    def transform(self, documents):
        return [
            ' '.join(self.normalize(doc)) for doc in documents
        ]

### Preprocess

In [5]:
class Preprocessor(object):
    """
    The preprocessor wraps a SqliteCorpusReader and manages the stateful
    tokenization and part of speech tagging into a directory that is stored
    in a format that can be read by the `PickledCorpusReader`. This format
    is more compact and necessarily removes a variety of fields from the
    document that are stored in Sqlite database. This format however is more
    easily accessed for common parsing activity.
    """

    def __init__(self, corpus, target=None, **kwargs):
        """
        The corpus is the `SqliteCorpusReader` to preprocess and pickle.
        The target is the directory on disk to output the pickled corpus to.
        """
        self.tasks = mp.cpu_count()
        self.corpus = corpus
        self.target = target

    @property
    def target(self):
        return self._target

    @target.setter
    def target(self, path):
        if path is not None:
            # Normalize the path and make it absolute
            path = os.path.expanduser(path)
            path = os.path.expandvars(path)
            path = os.path.abspath(path)

            if os.path.exists(path):
                if not os.path.isdir(path):
                    raise ValueError(
                        "Please supply a directory to write preprocessed data to."
                    )

        self._target = path

    def abspath(self, name):
        """
        Returns the absolute path to the target fileid from the corpus fileid.
        """
        # Create the pickle file extension
        fname  = str(name) + '.pickle'

        # Return the path to the file relative to the target.
        return os.path.normpath(os.path.join(self.target, fname))

    def tokenize(self, text):
        """
        Segments, tokenizes, and tags a document in the corpus. Returns a
        generator of paragraphs, which are lists of sentences, which in turn
        are lists of part of speech tagged words.
        """
        yield [
            nltk.pos_tag(nltk.wordpunct_tokenize(sent))
            for sent in nltk.sent_tokenize(text)
        ]

    def process(self, score_album_artist_text):
        """
        For a single file does the following preprocessing work:
            1. Checks the location on disk to make sure no errors occur.
            2. Gets all paragraphs for the given text.
            3. Segments the paragraphs with the sent_tokenizer
            4. Tokenizes the sentences with the wordpunct_tokenizer
            5. Tags the sentences using the default pos_tagger
            6. Writes the document as a pickle to the target location.
        This method is called multiple times from the transform runner.
        """
        score, album, artist, text = score_album_artist_text

        # Compute the outpath to write the file to.
        if album:
            name = album+'-'+artist
        else:
            name = artist
        target = self.abspath(slugify(name))
        parent = os.path.dirname(target)

        # Make sure the directory exists
        if not os.path.exists(parent):
            os.makedirs(parent)

        # Make sure that the parent is a directory and not a file
        if not os.path.isdir(parent):
            raise ValueError(
                "Please supply a directory to write preprocessed data to."
            )

        # Create a data structure for the pickle
        document = list(self.tokenize(text))
        document.append(score)

        # Open and serialize the pickle to disk
        with open(target, 'wb') as f:
            pickle.dump(document, f, pickle.HIGHEST_PROTOCOL)

        # Clean up the document
        del document

        # Return the target fileid
        return target

    def transform(self):
        """
        Transform the wrapped corpus, writing out the segmented, tokenized,
        and part of speech tagged corpus as a pickle to the target directory.
        This method will also directly copy files that are in the corpus.root
        directory that are not matched by the corpus.fileids().
        """
        # Make the target directory if it doesn't already exist
        if not os.path.exists(self.target):
            os.makedirs(self.target)

        for score_album_artist_text in self.corpus.scores_albums_artists_texts():
            yield self.process(score_album_artist_text)


class ParallelPreprocessor(Preprocessor):
    """
    Implements multiprocessing to speed up the preprocessing efforts.
    """

    def __init__(self, *args, **kwargs):
        """
        Get parallel-specific arguments and then call super.
        """
        self.tasks = mp.cpu_count()
        super(ParallelPreprocessor, self).__init__(*args, **kwargs)

    def on_result(self, result):
        """
        Appends the results to the master results list.
        """
        self.results.append(result)

    def transform(self):
        """
        Create a pool using the multiprocessing library, passing in
        the number of cores available to set the desired number of
        processes.
        """
        # Make the target directory if it doesn't already exist
        if not os.path.exists(self.target):
            os.makedirs(self.target)

        # Reset the results
        self.results = []

        # Create a multiprocessing pool
        pool  = mp.Pool(processes=self.tasks)
        tasks = [
            pool.apply_async(self.process, (score_album_artist_text,), callback=self.on_result)
            for score_album_artist_text in self.corpus.scores_albums_artists_texts()
        ]

        # Close the pool and join
        pool.close()
        pool.join()

        return self.results

In [6]:
%%time
## This is slow(ish) - takes c. an hour and a half
# Download the data from https://www.kaggle.com/nolanbconaway/pitchfork-data/data
# Preprocess the reviews corpus
if preproc:
    corpus = SqliteCorpusReader(sql_file)
    transformer = Preprocessor(corpus, pickle_dir)
    docs = transformer.transform()
    print(len(list(docs)))

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 7.87 µs


### Cross-validate review sentiment

In [14]:
# Scoring function and utilities
def documents(corpus):
    return list(corpus.reviews())

def make_continuous(corpus):
    return list(corpus.scores())

def make_categorical(corpus):
    """
    terrible : 0.0 < y <= 3.0
    okay     : 3.0 < y <= 5.0
    great    : 5.0 < y <= 7.0
    amazing  : 7.0 < y <= 10.1
    :param corpus:
    :return:
    """
    return np.digitize(make_continuous(corpus), [0.0, 3.0, 5.0, 7.0, 10.1])

def train_model(path, model, continuous=True, saveto=None, cv=4):
    """
    Trains model from corpus at specified path; constructing cross-validation
    scores using the cv parameter, then fitting the model on the full data and
    writing it to disk at the saveto path if specified. Returns the scores.
    """
    # Load the corpus data and labels for classification
    corpus = PickledReviewsReader(path)
    X = documents(corpus)
    if continuous:
        y = make_continuous(corpus)
        scoring = 'r2'
    else:
        y = make_categorical(corpus)
        scoring = 'f1_weighted'

    # Compute cross validation scores
    scores = cross_val_score(model, X, y, cv=cv, scoring=scoring)

    # Write to disk if specified
    if saveto:
        # Fit the model on entire data set
        model.fit(X, y)
        joblib.dump(model, saveto)

    # Return scores as well as training time via decorator
    return scores

In [8]:
%%time
# Set up processed review reader
reader = PickledReviewsReader(pickle_dir)

# Perform sentiment scoring
pipeline = Pipeline([
    ('norm', TextNormalizer()),
    ('tfidf', TfidfVectorizer()),
    ('ann', MLPClassifier(hidden_layer_sizes=[500,150], verbose=True))
])

print("Start training discreet model ...")
scores = train_model(pickle_dir, pipeline, continuous=False, saveto=pitchfork_model)

print("Training complete.")
for idx, score in enumerate(scores):
    print("Accuracy on slice #{}: {}.".format((idx+1), score))
print("Model saved to {}.".format(pitchfork_model))

Start training discreet model ...
Iteration 1, loss = 0.93580851
Iteration 2, loss = 0.49681042
Iteration 3, loss = 0.13560381
Iteration 4, loss = 0.02530439
Iteration 5, loss = 0.00604412
Iteration 6, loss = 0.00383449
Iteration 7, loss = 0.00323782
Iteration 8, loss = 0.00289548
Iteration 9, loss = 0.00268886
Iteration 10, loss = 0.00253897
Iteration 11, loss = 0.00245177
Iteration 12, loss = 0.00235821
Iteration 13, loss = 0.00230173
Iteration 14, loss = 0.00226554
Iteration 15, loss = 0.00217869
Iteration 16, loss = 0.00212844
Iteration 17, loss = 0.00208369
Iteration 18, loss = 0.00203649
Iteration 19, loss = 0.00198492
Iteration 20, loss = 0.00193895
Iteration 21, loss = 0.00190184
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 1, loss = 0.89090957
Iteration 2, loss = 0.42506844
Iteration 3, loss = 0.08429308
Iteration 4, loss = 0.01512366
Iteration 5, loss = 0.00512944
Iteration 6, loss = 0.00368741
Iteration 7, loss = 0.00317

In [17]:
%%time
# Ditto, continuous (regression) model

pipeline_reg = Pipeline([
    ('norm', TextNormalizer()),
    ('tfidf', TfidfVectorizer()),
    ('ann', MLPRegressor(hidden_layer_sizes=[500,150], verbose=True))
])

print("Start training regression model ...")
scores = train_model(pickle_dir, pipeline_reg, continuous=True, saveto=None)

print("Training complete.")
for idx, score in enumerate(scores):
    print("Accuracy on slice #{}: {}.".format((idx+1), score))

Start training regression model ...
Iteration 1, loss = 7.29345703
Iteration 2, loss = 0.82019703
Iteration 3, loss = 0.42778504
Iteration 4, loss = 0.19981920
Iteration 5, loss = 0.08335059
Iteration 6, loss = 0.03419112
Iteration 7, loss = 0.01846326
Iteration 8, loss = 0.01442684
Iteration 9, loss = 0.01388839
Iteration 10, loss = 0.01423167
Iteration 11, loss = 0.01470336
Iteration 12, loss = 0.01510508
Iteration 13, loss = 0.01523300
Iteration 14, loss = 0.01516401
Iteration 15, loss = 0.01513121
Iteration 16, loss = 0.01534073
Iteration 17, loss = 0.01543284
Iteration 18, loss = 0.01519092
Iteration 19, loss = 0.01506917
Iteration 20, loss = 0.01520778
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 1, loss = 5.96033903
Iteration 2, loss = 0.74651895
Iteration 3, loss = 0.37564046
Iteration 4, loss = 0.17460223
Iteration 5, loss = 0.07300537
Iteration 6, loss = 0.03271803
Iteration 7, loss = 0.02076787
Iteration 8, loss = 0.0178

## Deep learning with Keras and TensorFlow

In [10]:
# Functions and settings
N_FEATURES = 10000
DOC_LEN = 500
N_CLASSES = 4

def build_nn():
    """
    Create a function that returns a compiled neural network
    :return: compiled Keras neural network model
    """
    nn = Sequential()
    nn.add(Dense(500, activation='relu', input_shape=(N_FEATURES,)))
    nn.add(Dense(150, activation='relu'))
    nn.add(Dense(N_CLASSES, activation='softmax'))
    nn.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    return nn

def build_lstm():
    """
    LSTM alternative
    """
    lstm = Sequential()
    lstm.add(Embedding(N_FEATURES+1, 128, input_length=DOC_LEN))
    lstm.add(Dropout(0.4))
    lstm.add(LSTM(units=200, recurrent_dropout=0.2, dropout=0.2))
    lstm.add(Dropout(0.2))
    lstm.add(Dense(N_CLASSES, activation='sigmoid'))
    lstm.compile(
        loss='categorical_crossentropy', # b/c target vals are 1 or 2
        optimizer='adam',
        metrics=['accuracy']
    )
    return lstm

def train_nn_model(path, model, reader, saveto=None, cv=12, **kwargs):
    """
    Trains model from corpus at specified path;
    fitting the model on the full data and
    writing it to disk at the saveto directory if specified.
    Returns the scores.
    """
    # Load the corpus data and labels for classification
    corpus = reader(path)
    X = documents(corpus)
    y = make_categorical(corpus) # for Pitchfork

    # Compute cross validation scores
    # mp note: http://scikit-learn.org/stable/faq.html#why-do-i-sometime-get-a-crash-freeze-with-n-jobs-1-under-osx-or-linux
    scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')

    # Write to disk if specified
    if saveto:
        # Fit the model on entire data set
        model.fit(X, y)
        # have to save the keras part using keras' save method
        model.steps[-1][1].model.save(saveto['keras_model'])
        model.steps.pop(-1)
        # ... and use joblib to save the rest of the pipeline
        joblib.dump(model, saveto['sklearn_pipe'])

    # Return scores as well as training time via decorator
    return scores

In [11]:
%%time
# Work around buggy OpenMP linking on MacOS
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Build a Keras Sequential model for the Pitchfork reviews
cpath = pickle_dir
mpath = {
    'keras_model'  : os.path.join(pitchfork_dir, 'keras_nn.h5'),
    'sklearn_pipe' : os.path.join(pitchfork_dir, 'nn_pipeline.pkl')
}

pipeline = Pipeline([
    ('norm', TextNormalizer()),
    ('vect', TfidfVectorizer(max_features=N_FEATURES)), # need to control feature count
    ('nn', KerasClassifier(build_fn=build_nn, # pass but don't call the function!
                           epochs=10,
                           batch_size=128))
])

scores = train_nn_model(cpath, pipeline, PickledReviewsReader, saveto=mpath, cv=4)
for idx, score in enumerate(scores):
    print('Accuracy on slice #{}: {}.'.format((idx+1), score))
print('Model saved to {}.'.format(list(mpath)))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
10789/10789 [==============================] - 22s 2ms/step - loss: 0.8210 - acc: 0.6506
Epoch 2/10
10789/10789 [==============================] - 16s 1ms/step - loss: 0.4618 - acc: 0.8145
Epoch 3/10
10789/10789 [==============================] - 16s 2ms/step - loss: 0.1986 - acc: 0.9321
Epoch 4/10
10789/10789 [==============================] - 17s 2ms/step - loss: 0.0536 - acc: 0.9892
Epoch 5/10
10789/10789 [==============================] - 16s 1ms/step - loss: 0.0105 - acc: 0.9986
Epoch 6/10
10789/10789 [==============================] - 20s 2ms/step - loss: 0.0035 - acc: 0.9994
Epoch 7/10
10789/10789 [==============================] - 21s 2ms/step - loss: 0.0022 - acc: 0.9994
Epoch 8/10
10789/10789 [==============================] - 17s 2ms/step - loss: 0.0015 - acc: 0.9995
Epoch 9/10
10789/10789 [==============================] - 16s 2ms/step - loss: 0